In [ ]:
import numpy as np
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from tqdm import tqdm
import time

In [ ]:
# Число частиц
N = 60
N_values = np.arange(int(N - 0.05 * N), int(N + 0.05 * N)+1)
# Размер двумерной области
x_size = 10
y_size = 10
# Масса одной частицы
m = 0.2
sigma = 0.03
epsilon = 0.05
r_min = 0.03
kBT = 0.1
# Число степеней свободы для частицы
f = 2
# Постоянная Больцмана
kB = 1
T = kBT/kB
T_values = np.round(np.linspace((kBT - 0.1 * kBT), (kBT + 0.1 * kBT), 3), 2)

In [ ]:
print(N_values)

In [ ]:
print(T_values)

In [ ]:
def modified_potential_energy(positions, r_end, forces, N):
    energy = 0.0
    energy_end = 4 * epsilon * ((sigma / r_end)**12 - (sigma / r_end)**6)
    for i in range(N):
        for j in range(i + 1, N):
            r_vec = positions[i] - positions[j]
            r_vec[1] -= y_size * np.round(r_vec[1] / y_size)  # Периодические условия по оси y
            r = np.linalg.norm(r_vec)
            if r < r_min:  
                energy += (((4 * epsilon * ((sigma / r_min)**12 - (sigma / r_min)**6)) + np.dot(forces[i], forces[j])*(r_min-r)) - energy_end)
            elif r < r_end:
                energy += (4 * epsilon * ((sigma / r)**12 - (sigma / r)**6) - energy_end)
            else:
                continue
        if positions[i, 0] < r_end and positions[i, 0] > r_min: # вблизи левой стенки
            r = positions[i, 0]
            energy += (4 * epsilon * ((sigma / r)**12 - (sigma / r)**6) - energy_end)
        elif positions[i, 0] < r_end and positions[i, 0] < r_min:
            energy += (4 * epsilon * ((sigma / r_min)**12 - (sigma / r_min)**6) + (4 * epsilon * ((12 * (sigma / r_min)**12) - (6 * (sigma / r_min)**6))*(r_min-r)) - energy_end)
            
        elif positions[i, 0] > x_size - r_end and positions[i, 0] < x_size - r_min: # вблизи правой стенки
            r = x_size - positions[i, 0]
            energy += (4 * epsilon * ((sigma / r)**12 - (sigma / r)**6) - energy_end)
        elif positions[i, 0] > x_size - r_end and positions[i, 0] > x_size - r_min:
            energy += (4 * epsilon * ((sigma / r_min)**12 - (sigma / r_min)**6) + (4 * epsilon * ((12 * (sigma / r_min)**12) - (6 * (sigma / r_min)**6))*(r_min-r)) - energy_end)
        else:
            continue
    return energy

In [ ]:
def modified_forces_calc(positions, r_end):
    force = np.zeros_like(positions)
    virial = 0.0  

    for i in range(N):
        for j in range(i + 1, N):
            r_vec = positions[i] - positions[j]
            r_vec[1] -= y_size * np.round(r_vec[1] / y_size)  # Периодические условия по оси y
            r = np.linalg.norm(r_vec)
            if r < r_min:
                f_mag = 4 * epsilon * (12 * (sigma / r_min)**12 - 6 * (sigma / r_min)**6)
            elif r < r_end:
                f_mag = 4 * epsilon * (12 * (sigma / r)**12 - 6 * (sigma / r)**6)
            else:
                continue

            f_vec = f_mag * (r_vec / r)
            force[i] += f_vec
            force[j] -= f_vec

            virial += np.dot(r_vec, f_vec)

    
        f_mag_x = 0.0
        # Левая стенка
        if positions[i, 0] < r_end and positions[i, 0] > r_min:
            r = positions[i, 0]
            f_mag_x = 4 * epsilon * (12 * (sigma / r)**12 - 6 * (sigma / r)**6)
            force[i, 0] += f_mag_x
            virial += positions[i, 0] * f_mag_x  # r_x * F_x

        elif positions[i, 0] < r_end and positions[i, 0] <= r_min:
            r = r_min
            f_mag_x = 4 * epsilon * (12 * (sigma / r)**12 - 6 * (sigma / r)**6)
            force[i, 0] += f_mag_x
            virial += positions[i, 0] * f_mag_x

        # Правая стенка
        elif positions[i, 0] > x_size - r_end and positions[i, 0] < x_size - r_min:
            r = x_size - positions[i, 0]
            f_mag_x = 4 * epsilon * (12 * (sigma / r)**12 - 6 * (sigma / r)**6)
            force[i, 0] -= f_mag_x
            virial += (positions[i, 0] - x_size) * (-f_mag_x)  # r_x * F_x

        elif positions[i, 0] >= x_size - r_min:
            r = r_min
            f_mag_x = 4 * epsilon * (12 * (sigma / r)**12 - 6 * (sigma / r)**6)
            force[i, 0] -= f_mag_x
            virial += (positions[i, 0] - x_size) * (-f_mag_x)

    return force, virial

In [ ]:
instant_pressures = []
dt = 0.1
r_end = 2.5 * sigma
timesteps = np.arange(0, 1001, dt)
#velocities_4 = velocities.copy()
#pos_4 = pos.copy()
V = x_size * y_size
trajectory = []


start_time = time.time()
for n in N_values:
    n_side_x = int(np.ceil(np.sqrt(n * x_size / y_size)))
    n_side_y = int(np.ceil(np.sqrt(n * y_size / x_size)))
    spacing_x = x_size / n_side_x
    spacing_y = y_size / n_side_y
    pos = []
    for i in range(n_side_x):
        for j in range(n_side_y):
            if len(pos) < n:
                x = i * spacing_x + spacing_x/2
                y = j * spacing_y + spacing_y/2
                pos.append([x, y])

    pos = np.array(pos)
    for tetta in T_values:
        velocities = np.random.normal(0, np.sqrt(tetta / m), (n, 2))
        velocities -= np.mean(velocities, axis=0)
        pos_copy = pos.copy()
        E_kin = np.sum(0.5 * m * velocities**2)
        temperature = (1 / (N * f * kB)) * E_kin
        for t in timesteps:
            
            forces, virial = modified_forces_calc(pos_copy, r_end, n)
            a = forces / m
            velocities_i = velocities + a * dt
            
            # Термостат Берендсена
            scale = np.sqrt(1 + (dt / 0.1) * (T / temperature - 1))
            velocities_i *= scale
            
            pos_i = pos_copy + ((velocities + velocities_i) / 2) * dt
            # Периодические условия по оси y
            pos_i[:, 1] %= y_size
            # Искусственные стенки по оси x
            pos_i[:, 0] = np.clip(pos_i[:, 0], 0.01, x_size-0.01)
            E_kin = np.sum(0.5 * m * velocities_i**2)
            temperature = (1 / (N * f * kB)) * E_kin
            if t % 100 == 0:
                pressure = (1 / (V * 3)) * ((E_kin * 2) + virial)
                instant_pressures.append(pressure)
    
            velocities = velocities_i
            pos_copy = pos_i
end_time = time.time()
elapsed_time = end_time - start_time
print('Время расчета: ', elapsed_time, ' секунд')
pressures = np.array(instant_pressures)
mean_pressure = np.mean(pressures)
std_pressure = np.std(pressures)
print('Неопределенность выходной величины(давления): '+str(std_pressure))
# Оценка производных чувствительности
dP_dT = np.mean((pressures - np.roll(pressures, shift=1)) / (T_values[1] - T_values[0]))
dP_dN = np.mean((pressures[1:] - pressures[:-1]) / (N_values[1] - N_values[0]))
print('Производная чувствительности dP/dT: ', dP_dT)
print('Производная чувствительности dP/dN: ', dP_dN)
